# GFK

In [6]:
%load_ext autoreload
%autoreload 2

In [8]:
#adaptacion
from utils.adaptacion import *

#clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[1]
dims = dimensions[dataset_name]

## Pruebas con el dataset Twitter

In [2]:
print tipo
print dataset_name
print dims
print data_path

gfk
twitter
2000
data


In [3]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

### Creación de modelos de adaptación.

In [4]:
#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega

size_dims = [int(dims/4), int(dims/2)]
n_subs = [10, 50, 100]

parameters = {
    'dims': size_dims,
    'n_subs': n_subs
}


print "Adaptando entre:"
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "%s - %s" % (src, tgt)
            X_tr = np.asarray(labeled[src]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            gfk, score = gfk_grid_search(parameters, X_tr, y_tr, X_ts)
            print "Mejor modelo: %.3f" % score
            
            
            gfk_save_path = os.path.join(models_path, tipo, "%s_%s_%s.pkl" % (dataset_name, src,tgt))
            print "Guardando modelo en %s" % gfk_save_path
            save_adapted_model(gfk, gfk_save_path)

print "Adaptaciones completadas"

Adaptando entre:
rio2016 - thevoice


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Mejor modelo: 0.749
Guardando modelo en models/gfk/twitter_rio2016_thevoice.pkl
thevoice - rio2016
Mejor modelo: 0.735
Guardando modelo en models/gfk/twitter_thevoice_rio2016.pkl
Adaptaciones completadas


### Pruebas de clasificación

In [20]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### GFK ####
            #############
            # se adaptan los dominios usando GFK
            print "Adaptando dominios..."
            best_model_path = os.path.join(models_path, tipo, "%s_%s_%s.pkl" % (dataset_name, src, tgt))
            gfk = load_best_model(best_model_path)
            
            
            X_tr = np.asarray(labeled[src]['X_tr'][:, :dims].todense())
            X_tr_a = transform_gfk(X_tr, gfk)
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            X_ts_a = transform_gfk(X_ts, gfk)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            clf = get_best_score(X_tr_a, y_tr, clasifier='KNeighbors')
            t_error = 1-clf.score(X_ts_a, y_ts)
            #t_error = gfk_compute_accuracy(gfk.G, X_tr, y_tr, X_ts, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "Pruebas completadas."

Tarea 1 de 2
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 2
Cargando modelo existente.
Adaptando dominios...
Pruebas completadas.


In [21]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,GFK,r->t,rio2016,thevoice,16.081594,43.340164,27.258570
1,GFK,t->r,thevoice,rio2016,20.687361,39.784700,19.097339


In [22]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/gfk/twitter.csv
Resultados guardados.
